In [1]:
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [2]:
## Load the ANN trained model, scaler pickle, onehot

model = load_model('model.h5')

## Load encoder and scaler

with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)

with open('onehot_encoder_geo.pkl', 'rb') as file:
    onehot_encoder_geo = pickle.load(file)

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)


In [16]:
input_data = {
    'CreditScore': 600,
    'Geography': "France",
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [24]:
## Encoding the features

geo_encode = onehot_encoder_geo.transform(pd.DataFrame([[input_data['Geography']]], columns=['Geography']))
geo_encode_df = pd.DataFrame(geo_encode, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encode_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [25]:
input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [26]:
input_df = pd.concat([input_df.reset_index(drop=True), geo_encode_df], axis=1)

In [27]:
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,France,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [28]:
## Encode catego var

input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,France,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [30]:
#Concat onehote encode data

input_df = input_df.drop("Geography",axis=1)

In [31]:
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [32]:
# Scaling the input data

input_scalled = scaler.transform(input_df)
input_scalled

array([[-0.53598516,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [35]:
## Predict Churn

preds = model.predict(input_scalled)
preds

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step


array([[0.02036504]], dtype=float32)

In [37]:
pred_proba = preds[0][0]
pred_proba

np.float32(0.020365037)

In [38]:
if pred_proba >= 0.5:
    print("The customer will churn")
else:
    print("The customer will not churn")

The customer will not churn
